### GCN and Cora

在该实现中您可以看到如下功能：
1. 下载 JID2 数据集
2. 使用 GeometricFlux 定义模型并进行训练

In this template you can finish the following functions:
1. Download the JID2 data set
2. Define and train the model using GeometricFlux

In [1]:
;wget "https://github.com/tczhangzhi/Julia-Deeplearning/releases/download/v0.0.1/cora_features.jld2"

In [2]:
;wget "https://github.com/tczhangzhi/Julia-Deeplearning/releases/download/v0.0.1/cora_graph.jld2"

In [3]:
;wget "https://github.com/tczhangzhi/Julia-Deeplearning/releases/download/v0.0.1/cora_labels.jld2"

In [4]:
using GeometricFlux
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using JLD2  # use v0.1.2
using Statistics: mean
using SparseArrays
using LightGraphs.SimpleGraphs
using LightGraphs: adjacency_matrix
using CuArrays

@load "cora_features.jld2" features
@load "cora_labels.jld2" labels
@load "cora_graph.jld2" g

num_nodes = 2708
num_features = 1433
hidden = 16
target_catg = 7
epochs = 20

## Preprocessing data
train_X = Float32.(features)  # dim: num_features * num_nodes
train_y = Float32.(labels)  # dim: target_catg * num_nodes

adj_mat = Matrix{Float32}(adjacency_matrix(g))

## Model
model = Chain(GCNConv(adj_mat, num_features=>hidden, relu),
              Dropout(0.5),
              GCNConv(adj_mat, hidden=>target_catg),
              softmax)

## Loss
loss(x, y) = crossentropy(model(x), y)
accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))

## Training
ps = Flux.params(model)
train_data = [(train_X, train_y)]
opt = ADAM(0.0001)
evalcb() = @show(accuracy(train_X, train_y))

for i = 1:epochs
    Flux.train!(loss, ps, train_data, opt, cb=throttle(evalcb, 10))
end

┌ Warning: Package GeometricFlux does not have LightGraphs in its dependencies:
│ - If you have GeometricFlux checked out for development and have
│   added LightGraphs as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with GeometricFlux
│ Loading LightGraphs into GeometricFlux from project dependency, future warnings for GeometricFlux are suppressed.
└ @ nothing nothing:909


accuracy(train_X, train_y) = 0.21565731166912852
accuracy(train_X, train_y) = 0.3068685376661743
accuracy(train_X, train_y) = 0.404357459379616
accuracy(train_X, train_y) = 0.49741506646971934
accuracy(train_X, train_y) = 0.569423929098966
accuracy(train_X, train_y) = 0.6347858197932054
accuracy(train_X, train_y) = 0.681314623338257
accuracy(train_X, train_y) = 0.7127031019202363
accuracy(train_X, train_y) = 0.7341211225997046
accuracy(train_X, train_y) = 0.7573855243722304
accuracy(train_X, train_y) = 0.7784342688330872
accuracy(train_X, train_y) = 0.7913589364844904
accuracy(train_X, train_y) = 0.8035450516986706
accuracy(train_X, train_y) = 0.8157311669128509
accuracy(train_X, train_y) = 0.8282865583456426
accuracy(train_X, train_y) = 0.8338257016248154
accuracy(train_X, train_y) = 0.8415805022156573
accuracy(train_X, train_y) = 0.8508124076809453
accuracy(train_X, train_y) = 0.8589364844903988
accuracy(train_X, train_y) = 0.8670605612998523
